# Capstone Project - The Battle of Neighborhoods : Global Consultants

**Introduction**

Global Consultants is a consulting company that helps/advise their customers who is planning to shift to a new location for settling down.People leave their current location for various reasons like job transfers or for a better living. One of the most important factors that needs to be considered while planning to shift to a new city/place is job opportunities,amenities or facilities provided by the place where you chose to live. We would be able to enjoy life if we have a steady income. We would be happier or satisfied in our residence or place, if there are variety of options to earn a living and enjoy life. Therefore, main aim of company is to predict the best options,by which customers can select a best location to earn a living or enjoy life.For example, if customer is planning to moving to California , we would like to predict the nearby town or place that provides opportunities to earn a living and all amenities for them to enjoy.

Based on the area a customer would like to shift, with the help of location data of that area and the categories of interest of the customer including basic amenities , we wil provide various options to the customer. This project aims to group the areas in a location as different clusters and make decisions based on it.

The Global Consultants Company would be able to provide helpful information for their customers and they would be able to pick the best place to live accordingly. The customers would be highly satisfied with the service the Company is providing and this would help to increase the reputation of the company and gain more customers.This will help make their life easy in a new city or country.

**Data Description**

One city will be analysed in this project. Based on the city, the latitude and longitude values will be obtained by using geopy module. The data used in this project is provided by Foursquare location data.The only limitation includes the data obtained from Foursquare. The city’s latitude and longitude values will be used to extract the venues data from Foursquare location data using explore endpoint. From the obtained results which is a JSON file, we will filter the columns and create a table/dataframe. The table or Dataframe will have various categories like park,restaurants,bars,..etc information provided by a particular city. Analyzing the various categories offered by the city, using kmeans clustering,we will choose best location that provides all facilities for the customer to shift to a new city or country.

**Import required libraries**

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

from sklearn.cluster import KMeans

print('Libraries imported.')

**Define Foursquare Credentials**

In [2]:
CLIENT_ID = 'UQYLU3SP4ZRGQ5WRZI2MJ2RPTB0XQY4GD3RC1L12PALWXRMW' # your Foursquare ID
CLIENT_SECRET = 'OUNX25AU0HD3KYJRKAV1BAFRQNZI4C1VOICTKZ1PN42CPMBD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UQYLU3SP4ZRGQ5WRZI2MJ2RPTB0XQY4GD3RC1L12PALWXRMW
CLIENT_SECRET:OUNX25AU0HD3KYJRKAV1BAFRQNZI4C1VOICTKZ1PN42CPMBD


**Define the city and get its latitude & longitude**

In [3]:
address = 'California, USA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of California are :',latitude, longitude)

The geograpical coordinate of California are : 36.7014631 -118.7559974


**Explore the venues around the city**

In [4]:
# Define the corresponding URL
radius=150000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=UQYLU3SP4ZRGQ5WRZI2MJ2RPTB0XQY4GD3RC1L12PALWXRMW&client_secret=OUNX25AU0HD3KYJRKAV1BAFRQNZI4C1VOICTKZ1PN42CPMBD&ll=36.7014631,-118.7559974&v=20180605&radius=150000&limit=100'

In [5]:
# Send the GET Request and examine the results
results = requests.get(url).json()
results
print('There are {} Venues.'.format(len(results['response']['groups'][0]['items'])))
items = results['response']['groups'][0]['items']
items[0]

There are 100 Venues.


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4c9034380b9e3704c709665e',
  'name': 'General Sherman Tree',
  'location': {'address': 'Sequoia National Park',
   'lat': 36.58168963841338,
   'lng': -118.75145625841009,
   'labeledLatLngs': [{'label': 'display',
     'lat': 36.58168963841338,
     'lng': -118.75145625841009}],
   'distance': 13339,
   'postalCode': '93262',
   'cc': 'US',
   'city': 'Sequoia',
   'state': 'CA',
   'country': 'United States',
   'formattedAddress': ['Sequoia National Park',
    'Sequoia, CA 93262',
    'United States']},
  'categories': [{'id': '52e81612bcbc57f1066b7a24',
    'name': 'Tree',
    'pluralName': 'Trees',
    'shortName': 'Tree',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/default_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4c903

In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

**Display Dataframe obtained**

In [8]:
dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,General Sherman Tree,Tree,Sequoia National Park,36.581690,-118.751456,"[{'label': 'display', 'lat': 36.58168963841338...",13339,93262,US,Sequoia,CA,United States,"[Sequoia National Park, Sequoia, CA 93262, Uni...",NaN,NaN,4c9034380b9e3704c709665e
1,Kings Canyon National Park,National Park,State Highway 180,36.740316,-118.963127,"[{'label': 'display', 'lat': 36.74031630118368...",18981,93633,US,Sierra,CA,United States,"[State Highway 180, Sierra, CA 93633, United S...",NaN,NaN,4da5f33393a01f42fad18fce
2,General Grant Tree,Tree,NaN,36.748025,-118.971573,"[{'label': 'display', 'lat': 36.748025, 'lng':...",19920,93628,US,Hume,CA,United States,"[Hume, CA 93628, United States]",NaN,NaN,50189051e4b06251be6d0465
3,Kings Canyon Outlook,Scenic Lookout,NaN,36.719582,-118.897462,"[{'label': 'display', 'lat': 36.71958230036856...",12784,93262,US,Sequoia National Park,CA,United States,"[Sequoia National Park, CA 93262, United States]",NaN,NaN,4fd515dee4b06a2d8dd57917
4,Moro Rock,Mountain,NaN,36.544133,-118.765223,"[{'label': 'display', 'lat': 36.54413327593281...",17533,93262,US,NaN,California,United States,"[CA 93262, United States]",NaN,NaN,543fd18d498e66a3b41d3880


In [9]:
dataframe_filtered.categories.unique()

array(['Tree', 'National Park', 'Scenic Lookout', 'Mountain',
       'Tourist Information Center', 'New American Restaurant',
       'Burger Joint', 'Gastropub', 'Historic Site',
       'American Restaurant', 'BBQ Joint', 'Liquor Store',
       'Sushi Restaurant', 'Warehouse Store', 'Factory',
       'Japanese Restaurant', 'Donut Shop', 'Coffee Shop', 'Pizza Place',
       'Mexican Restaurant', 'Bakery', 'Movie Theater', 'Pet Store',
       'Frozen Yogurt Shop', 'Italian Restaurant', 'Deli / Bodega',
       'Sports Bar', 'Grocery Store', 'Breakfast Spot', 'Ice Cream Shop',
       'Park', 'Water Park', 'Gym', 'Theater', 'Steakhouse', 'Market',
       'Brewery', 'Multiplex', 'Mediterranean Restaurant',
       'Garden Center', 'Zoo Exhibit', 'Bar', 'Indian Restaurant',
       'Fast Food Restaurant', 'Shopping Mall', 'Arts & Crafts Store',
       'Lounge', 'Zoo', 'Seafood Restaurant', 'Diner', 'Sandwich Place',
       'College Basketball Court'], dtype=object)

In [10]:
df1=dataframe_filtered.drop(['address','distance','labeledLatLngs','postalCode','cc','state','country','formattedAddress','crossStreet','id','neighborhood'], axis=1)
df1.head()

,name,categories,lat,lng,city
0,General Sherman Tree,Tree,36.581690,-118.751456,Sequoia
1,Kings Canyon National Park,National Park,36.740316,-118.963127,Sierra
2,General Grant Tree,Tree,36.748025,-118.971573,Hume
3,Kings Canyon Outlook,Scenic Lookout,36.719582,-118.897462,Sequoia National Park
4,Moro Rock,Mountain,36.544133,-118.765223,NaN


In [11]:
df2=df1.set_index('city')
df2.head()

,name,categories,lat,lng
city,,,,
Sequoia,General Sherman Tree,Tree,36.581690,-118.751456
Sierra,Kings Canyon National Park,National Park,36.740316,-118.963127
Hume,General Grant Tree,Tree,36.748025,-118.971573
Sequoia National Park,Kings Canyon Outlook,Scenic Lookout,36.719582,-118.897462
NaN,Moro Rock,Mountain,36.544133,-118.765223


In [12]:
df3=df2.reset_index()
df3.head()

,city,name,categories,lat,lng
0,Sequoia,General Sherman Tree,Tree,36.581690,-118.751456
1,Sierra,Kings Canyon National Park,National Park,36.740316,-118.963127
2,Hume,General Grant Tree,Tree,36.748025,-118.971573
3,Sequoia National Park,Kings Canyon Outlook,Scenic Lookout,36.719582,-118.897462
4,NaN,Moro Rock,Mountain,36.544133,-118.765223


In [13]:
venues_map1 = folium.Map(location=[latitude, longitude], zoom_start=8) # generate map centred around City


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='California',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map1)


# add popular spots to the map as blue circle markers
for lat, lng,city,categories in zip(df3.lat, df3.lng,df3.city,df3.categories):
    label = '{}, {}'.format(city, categories)
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False).add_to(venues_map1)

# display map
venues_map1

In [14]:
df3

,city,name,categories,lat,lng
0,Sequoia,General Sherman Tree,Tree,36.581690,-118.751456
1,Sierra,Kings Canyon National Park,National Park,36.740316,-118.963127
2,Hume,General Grant Tree,Tree,36.748025,-118.971573
3,Sequoia National Park,Kings Canyon Outlook,Scenic Lookout,36.719582,-118.897462
4,NaN,Moro Rock,Mountain,36.544133,-118.765223
...,...,...,...,...,...
95,Fresno,Save Mart Center,College Basketball Court,36.809800,-119.738573
96,Fresno,Chick-fil-A,Fast Food Restaurant,36.808194,-119.773622
97,Fresno,Whole Foods Market,Grocery Store,36.809290,-119.805587
98,Fresno,Mediterranean Grill,Mediterranean Restaurant,36.742080,-119.779576


Cleaning dataframe to apply kmeans clustering

In [15]:
df4=df3.drop(['city','name','categories'], axis=1)
df4.head()

,lat,lng
0,36.581690,-118.751456
1,36.740316,-118.963127
2,36.748025,-118.971573
3,36.719582,-118.897462
4,36.544133,-118.765223


**Apply k-means clustering to group the venues**

In [16]:
kclusters = 5
kmeans = KMeans(init="k-means++", n_clusters=5, n_init=12).fit(df4)
#check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 1], dtype=int32)

In [17]:
k_means_cluster_centers = kmeans.cluster_centers_
k_means_cluster_centers

array([[  36.32450014, -119.32184898],
       [  36.80818685, -119.73864269],
       [  37.32883618, -118.37864014],
       [  36.59488297, -118.84381387],
       [  36.63688628, -118.08870948]])

Add clustering labels to the cleaned dataframe

In [18]:
df5=df3
df5.insert(0,"Cluster labels",kmeans.labels_)
df5.head()

,Cluster labels,city,name,categories,lat,lng
0,3,Sequoia,General Sherman Tree,Tree,36.581690,-118.751456
1,3,Sierra,Kings Canyon National Park,National Park,36.740316,-118.963127
2,3,Hume,General Grant Tree,Tree,36.748025,-118.971573
3,3,Sequoia National Park,Kings Canyon Outlook,Scenic Lookout,36.719582,-118.897462
4,3,NaN,Moro Rock,Mountain,36.544133,-118.765223


In [19]:
dataframe_cluster0=df5.loc[df5['Cluster labels']==0]
dataframe_cluster1=df5.loc[df5['Cluster labels']==1]
dataframe_cluster2=df5.loc[df5['Cluster labels']==2]
dataframe_cluster3=df5.loc[df5['Cluster labels']==3]
dataframe_cluster4=df5.loc[df5['Cluster labels']==4]

In [21]:
dataframe_cluster0.head()


,Cluster labels,city,name,categories,lat,lng
10,0,Visalia,In-N-Out Burger,Burger Joint,36.313395,-119.314239
11,0,Visalia,Pita Kabob Gastropub,Gastropub,36.331853,-119.292628
14,0,Visalia,Vintage Press Restaurante,American Restaurant,36.331414,-119.297473
16,0,Visalia,BevMo!,Liquor Store,36.296188,-119.313095
17,0,Visalia,Gozen,Sushi Restaurant,36.297966,-119.303238


In [23]:
print('Cluster 0',dataframe_cluster0.shape)
print('Cluster 1',dataframe_cluster1.shape)
print('Cluster 2',dataframe_cluster2.shape)
print('Cluster 3',dataframe_cluster3.shape)
print('Cluster 4',dataframe_cluster4.shape)

Cluster 0 (22, 6)
Cluster 1 (61, 6)
Cluster 2 (5, 6)
Cluster 3 (9, 6)
Cluster 4 (3, 6)


In [24]:
dataframe_cluster1.head()

,Cluster labels,city,name,categories,lat,lng
9,1,Sanger,School House Restaurant & Tavern,New American Restaurant,36.719783,-119.448029
34,1,Selma,Sal's Mexican Restaurant,Mexican Restaurant,36.559580,-119.615024
36,1,Clovis,Mickey's Frozen Yogurt,Frozen Yogurt Shop,36.809055,-119.673506
39,1,Clovis,House of Juju,American Restaurant,36.824326,-119.701708
42,1,Fresno,Dog House Grill,Sports Bar,36.808232,-119.736589


In [25]:
dataframe_cluster2.head()

,Cluster labels,city,name,categories,lat,lng
15,2,Big Pine,Copper Top BBQ,BBQ Joint,37.168273,-118.289309
30,2,Bishop,Erick Schat’s Bakkery,Bakery,37.367858,-118.395590
38,2,Bishop,Mahogany Smoked Meats,Deli / Bodega,37.376012,-118.417570
40,2,Bishop,Holy Smoke BBQ,BBQ Joint,37.368106,-118.395267
59,2,Bishop,The Looney Bean,Coffee Shop,37.363933,-118.395465


In [26]:
dataframe_cluster3.head()

,Cluster labels,city,name,categories,lat,lng
0,3,Sequoia,General Sherman Tree,Tree,36.581690,-118.751456
1,3,Sierra,Kings Canyon National Park,National Park,36.740316,-118.963127
2,3,Hume,General Grant Tree,Tree,36.748025,-118.971573
3,3,Sequoia National Park,Kings Canyon Outlook,Scenic Lookout,36.719582,-118.897462
4,3,NaN,Moro Rock,Mountain,36.544133,-118.765223


In [27]:
dataframe_cluster4.head()

,Cluster labels,city,name,categories,lat,lng
12,4,Independence,Manzanar National Historic Site,Historic Site,36.727696,-118.147820
13,4,Lone Pine,Seasons,American Restaurant,36.604812,-118.062769
29,4,Lone Pine,Eastern Sierra InterAgency Visitor Center,Tourist Information Center,36.578151,-118.055540


Identify the unique categories in the Cluster dataframe.

In [28]:
dataframe_cluster0.categories.unique()

array(['Burger Joint', 'Gastropub', 'American Restaurant', 'Liquor Store',
       'Sushi Restaurant', 'Warehouse Store', 'Factory',
       'Japanese Restaurant', 'Donut Shop', 'Coffee Shop', 'Pizza Place',
       'Mexican Restaurant', 'Movie Theater', 'Pet Store',
       'Italian Restaurant'], dtype=object)

In [29]:
dataframe_cluster1.categories.unique()

array(['New American Restaurant', 'Mexican Restaurant',
       'Frozen Yogurt Shop', 'American Restaurant', 'Sports Bar',
       'Grocery Store', 'Pizza Place', 'Sushi Restaurant',
       'Breakfast Spot', 'Burger Joint', 'Ice Cream Shop', 'Bakery',
       'Coffee Shop', 'Park', 'Water Park', 'Gym', 'Theater',
       'Steakhouse', 'Market', 'Brewery', 'Multiplex',
       'Mediterranean Restaurant', 'Garden Center', 'Zoo Exhibit', 'Bar',
       'Indian Restaurant', 'Fast Food Restaurant', 'Pet Store',
       'Gastropub', 'Shopping Mall', 'Arts & Crafts Store', 'Lounge',
       'Italian Restaurant', 'Zoo', 'Seafood Restaurant', 'Diner',
       'Sandwich Place', 'College Basketball Court'], dtype=object)

In [30]:
dataframe_cluster2.categories.unique()

array(['BBQ Joint', 'Bakery', 'Deli / Bodega', 'Coffee Shop'],
      dtype=object)

In [31]:
dataframe_cluster3.categories.unique()

array(['Tree', 'National Park', 'Scenic Lookout', 'Mountain',
       'Tourist Information Center'], dtype=object)

In [32]:
dataframe_cluster4.categories.unique()

array(['Historic Site', 'American Restaurant',
       'Tourist Information Center'], dtype=object)

**Generate map to visualize the Clusters**

Centroids of cluster is displayed as a black circle mark. Spots in cluster 0 is displayed in blue. Spots in cluster 1 is displayed in red. Spots in cluster 2 is displayed in green.Spots in cluster 3 is displayed in yellow.Spots in cluster 4 is displayed in white.

In [41]:
california_map = folium.Map(location=[latitude, longitude], zoom_start=8) # generate map centred around City

for lat, lng in (k_means_cluster_centers):
# add centroids of cluster as a black circle mark
    folium.CircleMarker(
    [lat, lng],
    radius=6,
    popup='Cluster_Center',
    fill=True,
    color='black',
    fill_color='black',
    fill_opacity=0.6
    ).add_to(california_map)


# add spots in cluster 0 to the map as blue circle markers
for lat, lng, city, categories in zip(dataframe_cluster0['lat'], dataframe_cluster0['lng'], 
                                           dataframe_cluster0['city'], dataframe_cluster0['categories']):
    label = '{}, {}'.format(city, categories)
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(california_map)

# add spots in cluster 1 to the map as red circle markers
for lat, lng, city, categories in zip(dataframe_cluster1['lat'], dataframe_cluster1['lng'], 
                                           dataframe_cluster1['city'], dataframe_cluster1['categories']):
    label = '{}, {}'.format(city, categories)
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(california_map)
    
# add spots in cluster 3 to the map as green circle markers
for lat, lng, city, categories in zip(dataframe_cluster2['lat'], dataframe_cluster2['lng'], 
                                           dataframe_cluster2['city'], dataframe_cluster2['categories']):
    label = '{}, {}'.format(city, categories)
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(california_map) 
# add spots in cluster 3 to the map as yellow circle markers
for lat, lng, city, categories in zip(dataframe_cluster3['lat'], dataframe_cluster3['lng'], 
                                           dataframe_cluster3['city'], dataframe_cluster3['categories']):
    label = '{}, {}'.format(city, categories)
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='yellow',
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(california_map) 
# add spots in cluster 4 to the map as white circle markers
for lat, lng, city, categories in zip(dataframe_cluster4['lat'], dataframe_cluster4['lng'], 
                                           dataframe_cluster4['city'], dataframe_cluster4['categories']):
    label = '{}, {}'.format(city, categories)
    label = folium.Popup(label, parse_html=True)    
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='white',
        fill_color='white',
        fill_opacity=0.7,
        parse_html=False).add_to(california_map)     
# display map
california_map

**Conclusion**

From the map and dataframe it is clear that the maximum number of restaurants,movie theatres,grocery shops,coffee shops,shopping malls etc are available in cluster1.One shifting to cluster1 can easily find a earn of living and spend life as other facilities are available. Cluster2 can be ignored as there are only three venues and they are bakery,BBQ and Coffee Shop. Cluster0 is second best option where we can find many restaurants,movie theatres and shops.Cluster 3 should be ignored because there is only national park,mountain and scenic which cannot be chosen as a place of living. It can be better chosen as tourist spot.Cluster 4 can be ignored as there is only historic site and tourist information centre.